In [1]:
from vpython import*
import math
import matplotlib.pyplot as plt
import numpy as np

<IPython.core.display.Javascript object>

In [82]:
scene1=canvas()

<IPython.core.display.Javascript object>

In [83]:
N = 3 # N by N by N array of atoms
k = 1
m = 1
spacing = 1
atom_radius = 0.3*spacing
L0 = spacing-1.8*atom_radius
V0 = pi*(0.5*atom_radius)**2*L0 # initial volume of spring
scene1.center = 0.5*(N-1)*vector(1,1,1)
dt = 0.04*(2*pi*sqrt(m/k))
axes = [vector(1,0,0), vector(0,1,0), vector(0,0,1)]

class Lattice():
    
    def __init__(self, momentumRange, x, y, z):
        atom = sphere()
        atom.pos = vector(x,y,z)*spacing
        atom.radius = atom_radius
        atom.color = vector(0,0.58,0.69)
        atom.momentum = momentumRange*vec.random()
    
lattice = np.array([[[Lattice(0.1*spacing*sqrt(k/m), x, y, z) for x in range(N)] 
                    for y in range(N)] for z in range(N)], dtype=object)

In [80]:
while(True):
    rate(60)
    for i in np.nditer(lattice, flags=['refs_ok']):
        i.atom.pos += (i.atom.momentum/m)*dt
    

AttributeError: 'numpy.ndarray' object has no attribute 'atom'